# Convert EBM to IRM

This notebook takes the three-layer energy balance model tunings from Donald Cummins and converts them to a three-layer impulse response function.

It will then save these into a CSV file.

In [ ]:
import os

import numpy as np
import pandas as pd
import scipy.linalg
from tqdm import tqdm

from fair21.constants.general import (
    EARTH_RADIUS,
    SECONDS_PER_YEAR,
    DOUBLING_TIME_1PCT
)
from fair21.defaults.gases import pre_industrial_concentration
from fair21.forcing.ghg import meinshausen

In [ ]:
df = pd.read_csv(
    os.path.join("..", "data", "calibration", "4xCO2_cummins.csv")
)

In [ ]:
models = df['model'].unique()
models

In [ ]:
params = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C3'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa3'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]

In [ ]:
params

In [ ]:
double_co2_concentration = pre_industrial_concentration.copy()
double_co2_concentration['CO2'] = pre_industrial_concentration['CO2'] * 2

quadruple_co2_concentration = pre_industrial_concentration.copy()
quadruple_co2_concentration['CO2'] = quadruple_co2_concentration['CO2'] * 4

rf_2co2, erf_2co2 = meinshausen(double_co2_concentration, pre_industrial_concentration)
rf_4co2, erf_4co2 = meinshausen(quadruple_co2_concentration, pre_industrial_concentration)

forcing_2co2_4co2_ratio=erf_2co2['CO2']/erf_4co2['CO2'] 
forcing_2co2_4co2_ratio# TODO: move to a constants module

forcing_2co2_4co2_ratio=0.476304  # TODO: un-hardcode this and calculate directly from Meinshausen or Etminan relations.

In [ ]:
class EnergyBalanceModel:
    
    # Energy balance basics
    ocean_heat_capacity = np.array([5, 20, 100])
    ocean_heat_transfer = np.array([1, 2, 1])
    deep_ocean_efficacy = 1

    # Tuning
    forcing_4co2 = 8
    
    # Stochastic parameters (see Cummins et al. 2020)
    stochastic_run = False
    sigma_eta = 0.5
    sigma_xi = 0.5
    gamma_autocorrelation = 2

    # Output - not yet relevant
#    start = 1850.0
#    end = 2101.0
#    timestep = 0.2
#    outtime = None,
#    forcing = np.zeros(2)
    
    def __init__(self, **kwargs):
        self.ocean_heat_capacity = kwargs.get('ocean_heat_capacity', self.ocean_heat_capacity)
        self.ocean_heat_transfer = kwargs.get('ocean_heat_transfer', self.ocean_heat_transfer)
        self.deep_ocean_efficacy = kwargs.get('deep_ocean_efficacy', self.deep_ocean_efficacy)
        self.forcing_4co2 = kwargs.get('forcing_4co2', self.forcing_4co2)
        self.stochastic_run = kwargs.get('stochastic_run', self.stochastic_run)
        self.sigma_eta = kwargs.get('sigma_eta', self.sigma_eta)
        self.sigma_xi = kwargs.get('sigma_xi', self.sigma_xi)
        self.gamma_autocorrelation = kwargs.get('gamma_autocorrelation', self.gamma_autocorrelation)
    
    def _eb_matrix(self):
        # Define the matrix of differential equations
        # Cummins et al. (2020); Leach et al. (2021)
        eb_matrix = np.array(
            [
                [
                    -(self.ocean_heat_transfer[0]+self.ocean_heat_transfer[1])/self.ocean_heat_capacity[0],
                    self.ocean_heat_transfer[1]/self.ocean_heat_capacity[0], 
                    0
                ],
                [
                    self.ocean_heat_transfer[1]/self.ocean_heat_capacity[1],
                    -(self.ocean_heat_transfer[1]+self.deep_ocean_efficacy*self.ocean_heat_transfer[2])/self.ocean_heat_capacity[1],
                    self.deep_ocean_efficacy*self.ocean_heat_transfer[2]/self.ocean_heat_capacity[1]
                ],
                [
                    0, 
                    self.ocean_heat_transfer[2]/self.ocean_heat_capacity[2],
                    -self.ocean_heat_transfer[2]/self.ocean_heat_capacity[2]
                ]
            ]
        )
        return(eb_matrix)
    
        
    def impulse_response(self):
        eb_matrix = self._eb_matrix()

        # calculate the eigenvectors and eigenvalues, these are the timescales of responses
        eb_matrix_eigenvalues, eb_matrix_eigenvectors = scipy.linalg.eig(eb_matrix)
        self.timescales = -1/(np.real(eb_matrix_eigenvalues))
        self.response_coefficients = self.timescales * (eb_matrix_eigenvectors[0,:] * scipy.linalg.inv(eb_matrix_eigenvectors)[:,0]) / self.ocean_heat_capacity[0]

    def emergent_parameters(self):
        # requires impulse response step
        if not hasattr(self, 'timescales'):
            self.impulse_response()
        self.ecs = self.forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(self.response_coefficients)
        self.tcr = self.forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(
            self.response_coefficients*(
                1 - self.timescales/DOUBLING_TIME_1PCT * (
                    1 - np.exp(-DOUBLING_TIME_1PCT/self.timescales)
                )
            )
        )

In [ ]:
ebm = EnergyBalanceModel(**params['ACCESS-CM2']['r1i1p1f1'])

In [ ]:
ebm.emergent_parameters()

In [ ]:
ebm.ecs

In [ ]:
for model in models:
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        ebm = EnergyBalanceModel(**params[model][run])
        ebm.emergent_parameters()
        params[model][run] = ebm.__dict__

In [ ]:
params

In [ ]:
# reconstruct a data table and save
df_out = pd.DataFrame(columns=['model', 'run', 'ecs', 'tcr', 'tau1', 'tau2', 'tau3', 'q1', 'q2', 'q3'])

#values_to_add = {'A': 1, 'B': 2}
#row_to_add = pd.Series(values_to_add, name='x')

#df = df.append(row_to_add)

count = 0
for model in models:
    for run in df.loc[df['model']==model, 'run']:
        values_to_add = {
            'model': model,
            'run': run,
            'ecs': params[model][run]['ecs'],
            'tcr': params[model][run]['tcr'],
            'tau1': params[model][run]['timescales'][0],
            'tau2': params[model][run]['timescales'][1],
            'tau3': params[model][run]['timescales'][2],
            'q1': params[model][run]['response_coefficients'][0],
            'q2': params[model][run]['response_coefficients'][1],
            'q3': params[model][run]['response_coefficients'][2],
        }
        row_to_add = pd.Series(values_to_add, name=count)
        df_out = df_out.append(row_to_add)
        count = count + 1

In [ ]:
df_out

In [ ]:
df_out.to_csv(os.path.join("..", "data", "calibration", "4xCO2_impulse_response.csv"))